### Edit this so that it's the month you want to generate:

In [ ]:
# To ZIP up all the reports from PythonAnywhere, use this command from a bash console
# zip -r the_month_you_want.zip data/

# Take all the CSV files you want to process (or, all the files for the month you want) and jam them in to /report
# Change these two values to the month you want

year = '2024'
month = '01'

# Do you only want the severe events?
severe = True

# Don't scroll away, but hit the double arrows above (or run every cell)
# Check to make sure your monthly report was generated 
# Move all files from /report to /archive (if you want to keep things tidy)


### Now hit the double right arrows in the menu above. The rest of this works on its own.

In [ ]:
import pandas as pd
from re import search, sub
from os import listdir 

def fill_or_swap_duplicates(final_df, new_df, column='nws_id'):
    if len(final_df) == 0:
        # If the final df is empty, fill it with the file's contents
        return new_df
    else:
        # Otherwise, remove all column values the final df has in common with the new file
        # and insert the newer values (assumes the files are in order).
        # This assumes the files are in alphabetical order, so careful
        result_df = final_df[~final_df[column].isin(new_df[column])]
        return pd.concat([result_df, new_df])


directory = 'report/'
salt = None
old_date, old_type = -1, None
final_df, final_events_df, final_locations_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

ugc_df = pd.read_csv('ugc_master.csv')
county_zip_df = pd.read_csv(
    'zip_code_database.csv',
    dtype = {
        'zip':str,
        'state':str,
        'ugc':str,
        'clean_county':str,
        'latitude':float,
        'longitude':float,
        'irs_estimated_population':int
        })
county_cleaner = (
    r"(North[ews]\w*|South[ews]\w*|Eastern|Western|Central|Interior|Coastal|"
    r"\s?Inland|\s?County|\s?Borough|Mountains?|Gorge|Area|Slopes|Upper|Lower)\s?")

file_matcher = (
    r"(?P<year>20\d{2})"
    r"(?P<month>\d{2})"
    r"(?P<date>\d{2})"
    r"_(?P<hour>\d{2}_)"
    r"(?P<filetype>events|event_locations)"
)

for file in sorted(listdir(directory)):
    # Make sure it's a filename format we expect
    # The result of this wille a dict with keys [year,month,date,hour,filetype]
    try: 
        match_dict = search(file_matcher,file).groupdict()
        if match_dict['year'] != year or match_dict['month'] != month:
            print('Found a file outside of file format, skipping',file)
            continue
    except:
        print('Failed to parse file name, skipping',file)
        continue
    
    # ensure the files are sorted
    # fill_or_swap_duplicates assumes the more recently viewed file is newer, 
    # and therefore has the most up-to-date information
    # So this ends up being important 
    match_dict['date'] = int(match_dict['date'])
    
    if old_date < 0:
        old_date = match_dict['date'] - 1
        old_type = match_dict['filetype']
    
    if old_type != match_dict['filetype'] and old_date == match_dict['date']:
        old_type = match_dict['filetype']
    elif old_date > match_dict['date']:
        assert False, "Files are not sorted! Files must be sorted alphanumerically."
    
    old_date = match_dict['date']
    
    filepath = directory + file 
    if match_dict['filetype'] == 'events':
        event_file_df = pd.read_csv(filepath, 
                              parse_dates = [2,3], 
                              date_parser=lambda col: pd.to_datetime(col, utc=True))
        final_events_df = fill_or_swap_duplicates(final_events_df, event_file_df)
    elif match_dict['filetype'] == 'event_locations':
        locations_file_df = pd.read_csv(filepath)
        final_locations_df = fill_or_swap_duplicates(final_locations_df, locations_file_df)
    else:
        print("I dunno what to do with this:",filepath)

In [ ]:
file

In [ ]:
# There are differences between the zip-county database and the NWS' county identification 
# This reconciles most of them, the ones that aren't caught tend to be geographic-specific locations 
ugc_df['clean_county'] = ugc_df['name'].apply(lambda x: sub(county_cleaner,'',x).strip())

# Build up a ugc-zip data table based on the UGCs we know so far 
ugc_zip = ugc_df.merge(
    county_zip_df, 
    left_on=['state','clean_county'], 
    right_on=['state','clean_county'],
    how='left')

# Rename, filter out to just the columns we need
ugc_zip = ugc_zip.rename(columns={'ugc_x':'ugc'})[['ugc','name','state','clean_county','zip','irs_estimated_population']]

# Make a table that joins our event_locations to this zip database
loc_zip = final_locations_df.merge(
    ugc_zip,
    left_on='ugc',
    right_on='ugc')

# Pandas assumes nothing about datatypes, so let's convert some stuff
loc_zip['nws_id'] = loc_zip['nws_id'].astype(str)
loc_zip['zip'] = loc_zip['zip'].astype(str)
final_events_df['nws_id'] = final_events_df['nws_id'].astype(str)

#One more gigantic merge 
prep_final_df = final_events_df.merge(
    loc_zip, 
    on='nws_id')

# Filter out events we don't have a zip for (because they're probably mountains or a gorge or whatever)
# Then filter out lesser events if Severe is True, otherwise bring in everyone 
final_df = ( prep_final_df[(~prep_final_df['zip'].isna()) &
                           (True if not(severe) else prep_final_df['severity'].isin(['Severe','Extreme']))]
           )

# Final columns 
print_df = final_df[[
    'clean_county',
    'state',
    'zip',
    'start',
    'end',
    'severity',
    'type',
    'snow_min',
    'snow_max',
    'ice_min',
    'ice_max',
    'sleet_min',
    'sleet_max',  
]]

print_df.reset_index(drop=True).to_csv("data/reports/{}{}_adverse_weather_report.csv".format(year,month),index=False)
print_df

In [ ]:
for row in print_df.groupby(['start_date','state']).count()['type'].reset_index().sort_values(['start_date', 'type'], ascending=[True,False]).itertuples():  
    print(f"{row.start_date} - {row.state} ({row.type})")